# Lab 19: Binary Analysis Basics

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab19_binary_basics.ipynb)

Essential binary/malware analysis concepts before building YARA rules.

## Learning Objectives
- Understand PE (Portable Executable) file structure
- Extract strings, imports, and sections from binaries
- Calculate file entropy (detect packing/encryption)
- Identify suspicious indicators in executables

**Next:** Lab 21 (YARA Generator)

In [ ]:
#@title Install dependencies (Colab only)
#@markdown Run this cell to install required packages in Colab

%pip install -q pefile

In [ ]:
import math
import re
from collections import Counter
from typing import List, Dict

print("✅ Libraries loaded!")

## Step 1: Understanding Entropy

**Entropy** measures randomness in data (0-8 bits per byte):

```
ENTROPY SCALE (0-8 bits per byte):
───────────────────────────────────────────────────
0-1   │███░░░░░░░░░░░░░░░│  Highly repetitive
1-4   │██████░░░░░░░░░░░░│  Plain text, code
4-6   │█████████░░░░░░░░░│  Normal executable
6-7   │████████████░░░░░░│  Compressed data
7-8   │███████████████░░░│  Encrypted/packed ⚠️
───────────────────────────────────────────────────
```

In [ ]:
def calculate_entropy(data: bytes) -> float:
    """
    Calculate Shannon entropy of binary data.

    Entropy measures how random/unpredictable data is.
    Higher entropy = more random = likely encrypted/compressed.

    Args:
        data: Binary data to analyze

    Returns:
        Entropy value (0-8 bits per byte)
    """
    if not data:
        return 0.0

    # Count byte frequencies
    counter = Counter(data)
    length = len(data)

    # Calculate Shannon entropy
    entropy = 0.0
    for count in counter.values():
        probability = count / length
        entropy -= probability * math.log2(probability)

    return entropy

# Test with different types of data
test_cases = [
    ("All zeros", b'\x00' * 1000),
    ("Plain text", b"Hello World! This is normal text content." * 20),
    ("Normal code", b"mov eax, ebx\x00call 0x401000\x00push ebp\x00" * 30),
    ("Random/encrypted", bytes([i % 256 for i in range(1000)])),
]

print("Entropy Analysis Examples:")
print("=" * 60)
for name, data in test_cases:
    ent = calculate_entropy(data)
    level = "⚠️ HIGH" if ent > 7.0 else "Normal" if ent > 4.0 else "Low"
    bar = "█" * int(ent * 2) + "░" * (16 - int(ent * 2))
    print(f"{name:20} | {bar} | {ent:.2f} ({level})")

## Step 2: Extracting Strings

Strings in binaries reveal:
- URLs and domains (C2 servers)
- Registry paths (persistence)
- File paths (targets)
- Commands (capabilities)

In [ ]:
def extract_strings(data: bytes, min_length: int = 4) -> List[str]:
    """
    Extract ASCII strings from binary data.

    Args:
        data: Binary data to search
        min_length: Minimum string length to extract

    Returns:
        List of extracted strings
    """
    # Pattern for printable ASCII characters
    pattern = rb'[\x20-\x7e]{' + str(min_length).encode() + rb',}'
    matches = re.findall(pattern, data)
    return [m.decode('ascii', errors='ignore') for m in matches]

# Simulated malware sample (fake data for demo)
fake_malware = b"""
MZ\x90\x00\x03\x00\x00\x00\x04\x00\x00\x00
http://evil-c2.malware.com/beacon
HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Run
cmd.exe /c whoami
powershell -enc JABjAGwAaQBlAG4AdA
C:\Users\victim\Documents\passwords.txt
SELECT * FROM credentials WHERE username='admin'
\x00\x00kernel32.dll\x00\x00VirtualAlloc\x00WriteProcessMemory
"""

print("🔍 STRING EXTRACTION")
print("=" * 60)

strings = extract_strings(fake_malware)
print(f"Found {len(strings)} strings:\n")

for s in strings:
    # Flag suspicious strings
    suspicious = any(kw in s.lower() for kw in [
        "http", "hkey", "cmd.exe", "powershell", "password",
        "virtualalloc", "writeprocess", "credentials"
    ])
    flag = "⚠️ " if suspicious else "   "
    print(f"{flag}{s[:60]}")

## Step 3: Suspicious API Imports

API imports reveal what a program can do!

```
🔴 HIGH RISK APIs (potential malware):
   VirtualAlloc, VirtualProtect    → Code injection
   CreateRemoteThread              → Process injection
   WriteProcessMemory              → Memory manipulation
   RegSetValueEx                   → Persistence
   InternetOpen, URLDownloadToFile → C2/download
   CryptEncrypt, CryptDecrypt      → Ransomware
```

In [ ]:
# Suspicious API categories
SUSPICIOUS_APIS = {
    "Process Injection": [
        "VirtualAlloc", "VirtualAllocEx", "VirtualProtect",
        "WriteProcessMemory", "CreateRemoteThread", "NtCreateThreadEx"
    ],
    "Credential Theft": [
        "CredRead", "CredEnumerate", "LsaRetrievePrivateData",
        "CryptUnprotectData"
    ],
    "Persistence": [
        "RegSetValueEx", "RegCreateKeyEx", "CreateService",
        "SetWindowsHookEx"
    ],
    "Network/C2": [
        "InternetOpen", "InternetConnect", "HttpOpenRequest",
        "URLDownloadToFile", "WinHttpOpen"
    ],
    "Encryption/Ransomware": [
        "CryptEncrypt", "CryptDecrypt", "CryptAcquireContext",
        "CryptGenKey"
    ],
    "Defense Evasion": [
        "IsDebuggerPresent", "CheckRemoteDebuggerPresent",
        "NtQueryInformationProcess", "GetTickCount"
    ],
}

def analyze_imports(imports: List[str]) -> Dict[str, List[str]]:
    """
    Categorize imports by suspicious activity type.

    Args:
        imports: List of API function names

    Returns:
        Dict mapping category to found suspicious APIs
    """
    findings = {}
    for category, apis in SUSPICIOUS_APIS.items():
        found = [api for api in apis if api in imports]
        if found:
            findings[category] = found
    return findings

# Simulated imports from a sample
sample_imports = [
    "GetModuleHandle", "LoadLibrary", "GetProcAddress",  # Normal
    "VirtualAlloc", "WriteProcessMemory", "CreateRemoteThread",  # Injection!
    "InternetOpen", "HttpOpenRequest",  # Network
    "RegSetValueEx",  # Persistence
    "CreateFile", "ReadFile", "WriteFile",  # Normal file ops
]

print("📦 IMPORT ANALYSIS")
print("=" * 60)
print(f"Total imports: {len(sample_imports)}\n")

findings = analyze_imports(sample_imports)
if findings:
    print("⚠️ SUSPICIOUS APIs DETECTED:\n")
    for category, apis in findings.items():
        print(f"  {category}:")
        for api in apis:
            print(f"    • {api}")
        print()
else:
    print("✅ No obviously suspicious APIs found")

## 🎉 Key Takeaways

1. **PE structure** - Know the header, sections, imports
2. **Entropy** - High entropy (>7) often means packed/encrypted
3. **Imports** - APIs reveal capabilities (injection, C2, persistence)
4. **Strings** - URLs, paths, commands are IOCs
5. **Sections** - Anomalies indicate packing/obfuscation

## Quick Reference: Red Flags

| Indicator | What It Suggests |
|-----------|------------------|
| Entropy > 7.0 | Packed or encrypted |
| VirtualAlloc + WriteProcessMemory | Code injection |
| InternetOpen + RegSetValueEx | C2 with persistence |
| CryptEncrypt + FindFirstFile | Potential ransomware |
| Encoded PowerShell commands | Obfuscated execution |

## Next Steps

- **Lab 21**: Generate YARA rules from these indicators
- **Lab 31**: Apply to ransomware detection
- **Lab 13**: Analyze memory dumps